In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import Reds9, Viridis256
from bokeh.models import Legend, LegendItem, DatetimeTickFormatter, LinearColorMapper
from bokeh.transform import linear_cmap

output_file('temp.html')

In [46]:
def plot_lines(data_file, output_file, depth_type, start_date, end_date, num_outliers):
    data = pd.read_csv(data_file)
    print(data.shape)
    data = data.set_index(pd.date_range(start=start_date, end=end_date).date)
    data = data.transpose()
    data = data.set_index(pd.date_range(start=start_date, periods=24, freq='H').time)

    data_outputs = pd.read_csv(output_file).set_index(pd.date_range(start=start_date, end=end_date).date)

    data_outputs = data_outputs.transpose()

    data_final = pd.concat([data,data_outputs])

    data_final = data_final.sort_values(by=depth_type,axis=1)
    #data_final.loc['tmd',:]
    
    data_outliers = data_final.iloc[:,:num_outliers]
    data_median   = data_final.iloc[:,data_final.shape[1]-1]
    data_top50   = data_final.iloc[:,(data_final.shape[1]//2):data_final.shape[1]-1]
    #data_middle   = data_final.iloc[:,num_outliers:data_final.shape[1]-1]
    data_ordered  = pd.concat([data_top50, data_outliers, data_median], axis=1)
    
    #color_list = list(data_final.loc["color",:])
    color_top50 = ["#fbb4b9"] * (data_ordered.shape[1] - (num_outliers+1))
    color_median = ["Blue"]
    color_outliers = ["Red"] * num_outliers
    color_list = color_top50 + color_outliers + color_median
    print(len(color_list))
    
    alpha_list = [0.5] * data_final.shape[1]
    alpha_list[0:num_outliers] = [1] * num_outliers
    alpha_list[len(alpha_list)-1] = 1

    numlines = len(data_ordered.columns)

    p = figure(width=1000, height=800, x_axis_type="datetime") 
    p.multi_line(xs=[data_ordered.index.values]*numlines,
                 ys=[data_ordered[name].values for name in data_ordered.iloc[0:24]],
                 line_color=color_list,
                 line_width=5)
    return p

In [47]:
plot_taxis_v2 = plot_lines('../data/taxis_v2.csv',"../outputs/taxis_v2_out.txt",'tmd','1/1/2016','31/12/2018', 10)
show(plot_taxis_v2)

(1096, 24)
558


In [49]:
plot_taxis_v1 = plot_lines('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'od','1/1/2018','31/12/2018', 10)
show(plot_taxis_v1)

(365, 24)
193


In [51]:
plot_airquality_v1 = plot_lines('../data/airquality_v1.csv',"../outputs/airquality_v1_out.txt",'fd','03/10/2004','04/04/2005',10)
show(plot_airquality_v1)

(391, 24)
206


In [168]:
data_outputs['td'].quantile([0.25,0.5,0.75])

0.25    0.013069
0.50    0.032895
0.75    0.074495
Name: td, dtype: float64

In [169]:
def get_color_from_depth(depth):
    '''
    if depth < 0.270425: return '#fef0d9'
    if depth < 0.353502: return '#fdcc8a'
    '''
    if depth < 0.074495: return '#fef0d9'
    else: return '#fdcc8a' 


data_outputs['color'] = data_outputs['td'].map(lambda x: get_color_from_depth(x))

In [11]:
len(pd.date_range(start='03/10/2004', end='04/04/2005').date)

391